### PROJETO FINAL



 ALUNOS:




 ESTE PROJETO ...

 - tittle, category:
  - linkar cada notícia a um ID
  - tokenizar 
  - representar de maneira abstrata (BERTimbau) -> Embedding


In [1]:
import os
import joblib
import sys
sys.path.append('..')


import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

from src.sentence_cleaner import SentenceCleaner
from src.data_scorer_calculator import DataScorerCalculator
from src.sentences_similarity_calculator import SentenceSimilarityCalculator

In [2]:
df_notices = pd.read_csv("../dataset/notices_small.csv", low_memory=False, encoding="UTF-8")
#df_notices = df_notices[df_notices['source'] != "CartaCapital"]
df_notices['publication_date'] = pd.to_datetime(df_notices['publication_date'], format='mixed', errors='coerce')
df_notices = df_notices.dropna(subset=['title','publication_date'])
df_notices = df_notices.drop_duplicates(subset=['title'])

In [3]:
df_notices.iloc[20002]

title                           Olímpia registra 2ª morte por dengue 
text                A informação foi confirmada nesta sexta-feira ...
category                           São José do Rio Preto e Araçatuba 
publication_date                           2024-05-03 18:23:10.210666
source                                                             G1
image               https://s2-g1.glbimg.com/YnndF2E6MQYLMYSMPZ9jz...
link                https://g1.globo.com/sp/sao-jose-do-rio-preto-...
Name: 20112, dtype: object

In [4]:
df_notices

,title,text,category,publication_date,source,image,link
0,Aeroporto de Porto Alegre suspende voos por te...,Concessionária pede que os passageiros entrem ...,sociedade,2024-05-03 21:33:00.000000,CartaCapital,https://www.cartacapital.com.br/wp-content/upl...,https://www.cartacapital.com.br/sociedade/aero...
1,Lula sanciona marco legal dos jogos eletrônico...,Não entram nesta categoria de gamer os jogos d...,politica,2024-05-03 21:15:00.000000,CartaCapital,https://www.cartacapital.com.br/wp-content/upl...,https://www.cartacapital.com.br/politica/lula-...
2,Como Zanin freia pedidos de entidades sobre a ...,A Corte está a um voto de confirmar a decisão ...,economia,2024-05-03 21:15:00.000000,CartaCapital,https://www.cartacapital.com.br/wp-content/upl...,https://www.cartacapital.com.br/economia/como-...
3,Governo Tarcísio usa vídeo do MBL em material ...,Material explica para que servem os grêmios es...,educacao,2024-05-03 20:52:00.000000,CartaCapital,https://www.cartacapital.com.br/wp-content/upl...,https://www.cartacapital.com.br/educacao/gover...
4,Justiça de São Paulo manda prender motorista d...,A decisão partiu do desembargador João Augusto...,sociedade,2024-05-03 20:26:00.000000,CartaCapital,https://www.cartacapital.com.br/wp-content/upl...,https://www.cartacapital.com.br/sociedade/just...
...,...,...,...,...,...,...,...
37012,Programação do Arraiá do Interior 2024 é divul...,Atividades acontecerão de 1º de junho a 26 de ...,São João na Paraíba,2024-04-03 22:26:33.777132,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/pb/paraiba/sao-joao/notic...
37013,Dupla é presa com tijolos de cocaína e mais de...,Suspeitos foram detidos durante patrulhamento ...,Sorocaba e Jundiaí,2024-04-03 22:26:33.781281,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/sp/sorocaba-jundiai/notic...
37014,VÍDEO: Mulher é assaltada dentro de carro a ca...,"Segundo o dono da empresa, a vítima do assalto...",Sul de Minas,2024-04-03 22:26:33.785399,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/mg/sul-de-minas/noticia/2...
37015,"Em Santarém, Sefa apreende mais de 66 mil garr...",Foram lavrados dois Termos de Apreensão e Depó...,Santarém e Região,2024-04-03 22:26:33.789469,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/pa/santarem-regiao/notici...


In [5]:
data_scorer_calculator = DataScorerCalculator()

In [6]:
df_notices['normalized_data_scores'] = data_scorer_calculator.calculate_data_score(df_notices['publication_date'])

In [7]:
df_notices

,title,text,category,publication_date,source,image,link,normalized_data_scores
0,Aeroporto de Porto Alegre suspende voos por te...,Concessionária pede que os passageiros entrem ...,sociedade,2024-05-03 21:33:00.000000,CartaCapital,https://www.cartacapital.com.br/wp-content/upl...,https://www.cartacapital.com.br/sociedade/aero...,0.000000
1,Lula sanciona marco legal dos jogos eletrônico...,Não entram nesta categoria de gamer os jogos d...,politica,2024-05-03 21:15:00.000000,CartaCapital,https://www.cartacapital.com.br/wp-content/upl...,https://www.cartacapital.com.br/politica/lula-...,0.000000
2,Como Zanin freia pedidos de entidades sobre a ...,A Corte está a um voto de confirmar a decisão ...,economia,2024-05-03 21:15:00.000000,CartaCapital,https://www.cartacapital.com.br/wp-content/upl...,https://www.cartacapital.com.br/economia/como-...,0.000000
3,Governo Tarcísio usa vídeo do MBL em material ...,Material explica para que servem os grêmios es...,educacao,2024-05-03 20:52:00.000000,CartaCapital,https://www.cartacapital.com.br/wp-content/upl...,https://www.cartacapital.com.br/educacao/gover...,0.000000
4,Justiça de São Paulo manda prender motorista d...,A decisão partiu do desembargador João Augusto...,sociedade,2024-05-03 20:26:00.000000,CartaCapital,https://www.cartacapital.com.br/wp-content/upl...,https://www.cartacapital.com.br/sociedade/just...,0.000000
...,...,...,...,...,...,...,...,...
37012,Programação do Arraiá do Interior 2024 é divul...,Atividades acontecerão de 1º de junho a 26 de ...,São João na Paraíba,2024-04-03 22:26:33.777132,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/pb/paraiba/sao-joao/notic...,0.065359
37013,Dupla é presa com tijolos de cocaína e mais de...,Suspeitos foram detidos durante patrulhamento ...,Sorocaba e Jundiaí,2024-04-03 22:26:33.781281,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/sp/sorocaba-jundiai/notic...,0.065359
37014,VÍDEO: Mulher é assaltada dentro de carro a ca...,"Segundo o dono da empresa, a vítima do assalto...",Sul de Minas,2024-04-03 22:26:33.785399,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/mg/sul-de-minas/noticia/2...,0.065359
37015,"Em Santarém, Sefa apreende mais de 66 mil garr...",Foram lavrados dois Termos de Apreensão e Depó...,Santarém e Região,2024-04-03 22:26:33.789469,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/pa/santarem-regiao/notici...,0.065359


##### Testar utilizando a coluna categories
 - remover categories insignificantes <= 10
 - adicionar ao titulo com a categoria repetida para ter mais peso

In [8]:
data_train = df_notices[['title', 'normalized_data_scores']].copy()

print('Training dataset size:', len(data_train))

Training dataset size: 36417


In [9]:
#data_train.loc[:, 'text_full']= data_train.loc[:, 'title'].str.cat(data_train['Content'], sep=' ')

In [10]:
data_train

,title,normalized_data_scores
0,Aeroporto de Porto Alegre suspende voos por te...,0.000000
1,Lula sanciona marco legal dos jogos eletrônico...,0.000000
2,Como Zanin freia pedidos de entidades sobre a ...,0.000000
3,Governo Tarcísio usa vídeo do MBL em material ...,0.000000
4,Justiça de São Paulo manda prender motorista d...,0.000000
...,...,...
37012,Programação do Arraiá do Interior 2024 é divul...,0.065359
37013,Dupla é presa com tijolos de cocaína e mais de...,0.065359
37014,VÍDEO: Mulher é assaltada dentro de carro a ca...,0.065359
37015,"Em Santarém, Sefa apreende mais de 66 mil garr...",0.065359


#### Limpeza e tratamento das sentenças

In [11]:
sentece_cleaner = SentenceCleaner()

In [12]:
data_train.loc[:, 'sentences'] = data_train.loc[:, 'title'].apply(sentece_cleaner.clear_sentence)

In [14]:
data_train['sentences'].values

array(['Aeroporto Porto Alegre suspende voos indeterminado',
       'Lula sanciona marco legal jogos eletrônicos Brasil',
       'Como Zanin freia pedidos entidades desoneração folha Fux interromper julgamento',
       ...,
       'VÍDEO Mulher assaltada carro agência bancária trevo MG dupla levou malote empresa',
       'Em Santarém Sefa apreende  garrafas bebida alcoólica  caixas balas',
       'Deputados autorizam empréstimo R$  milhões Governo Bahia detalhamento uso questionada'],
      dtype=object)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
tfidf_vectorizer = TfidfVectorizer()

In [41]:
tfidf_matrix = tfidf_vectorizer.fit_transform(data_train['sentences'].values)

In [42]:
tfidf_matrix

<36417x25398 sparse matrix of type '<class 'numpy.float64'>'
	with 291931 stored elements in Compressed Sparse Row format>

In [43]:
sentences_sym_calculator = SentenceSimilarityCalculator()

In [44]:
raw_sentence = 'Trump diz que foi “lindo ver” ação policial contra manifestantes pró-Palestina'
new_doc = sentece_cleaner.clear_sentence(raw_sentence)
new_sent = tfidf_vectorizer.transform([new_doc])

In [46]:
print(f'Alvo -> {raw_sentence}')
print('\nTop-5 similarity rate:\n')

for idx in sentences_sym_calculator.sym_by_date(new_sent, tfidf_matrix, data_train['normalized_data_scores'].values, 20, 5):
    print(f'{data_train["title"].values[idx]}')

Alvo -> Trump diz que foi “lindo ver” ação policial contra manifestantes pró-Palestina

Top-5 similarity rate:

Policial dispara acidentalmente dentro de prédio de universidade de Nova York ao retirar manifestantes pró-Palestina
Polícia entra na Universidade Columbia e prende manifestantes pró-Palestina 
Estudantes pró-Palestina acampam na maior universidade do México
Polícia expulsa manifestantes pró-palestinos de universidade de elite de Paris
Universidade de Columbia suspende manifestantes pró-Palestina que desafiaram ultimato


### Testando pipeline de funcionamento do sistema de recomendação:

In [1]:
import sys
sys.path.append('..')

import pandas as pd

from src.run_recomendation_system import RunRecomendationSystem

In [2]:
dataset = pd.read_csv("../dataset/notices_small.csv", low_memory=False, encoding="UTF-8")

In [3]:
dataset_origin = dataset.iloc[:-1000]
dataset_test = dataset.iloc[-1000:]

In [4]:
recomendation_system = RunRecomendationSystem(dataset_origin)

In [5]:
targets = dataset_test[-9:]

In [6]:
targets

,title,text,category,publication_date,source,image,link
37008,"Homem é morto com tiros no peito, boca e perna...",Nenhum suspeito foi identificado ou preso; cas...,São José do Rio Preto e Araçatuba,2024-04-03 22:26:33.756561,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/sp/sao-jose-do-rio-preto-...
37009,Policial militar é assassinado a tiros enquant...,Agente estava a caminho do quartel quando foi ...,Ceará,2024-04-03 22:26:33.763455,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/ce/ceara/noticia/2024/04/...
37010,Câmara de Taubaté aprova reajuste salarial de ...,Projeto foi aprovado por unanimidade pelos ver...,Vale do Paraíba e Região,2024-04-03 22:26:33.768573,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/sp/vale-do-paraiba-regiao...
37011,Suspeito de atropelar ex-vereador no interior ...,Vítima de 74 anos foi atropelada ao atravessar...,Bahia,2024-04-03 22:26:33.773026,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/ba/bahia/noticia/2024/04/...
37012,Programação do Arraiá do Interior 2024 é divul...,Atividades acontecerão de 1º de junho a 26 de ...,São João na Paraíba,2024-04-03 22:26:33.777132,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/pb/paraiba/sao-joao/notic...
37013,Dupla é presa com tijolos de cocaína e mais de...,Suspeitos foram detidos durante patrulhamento ...,Sorocaba e Jundiaí,2024-04-03 22:26:33.781281,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/sp/sorocaba-jundiai/notic...
37014,VÍDEO: Mulher é assaltada dentro de carro a ca...,"Segundo o dono da empresa, a vítima do assalto...",Sul de Minas,2024-04-03 22:26:33.785399,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/mg/sul-de-minas/noticia/2...
37015,"Em Santarém, Sefa apreende mais de 66 mil garr...",Foram lavrados dois Termos de Apreensão e Depó...,Santarém e Região,2024-04-03 22:26:33.789469,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/pa/santarem-regiao/notici...
37016,Deputados autorizam empréstimo de R$ 400 milhõ...,Parlamentar da situação apontou que verba será...,Bahia,2024-04-03 22:26:33.793424,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/ba/bahia/noticia/2024/04/...


In [7]:
dataset_sentences, dataset_recomendations = recomendation_system.run(targets, dataset_test[:-9])


In [8]:
dataset_sentences

,title,text,category,publication_date,source,image,link,sentences
37008,"Homem é morto com tiros no peito, boca e perna...",Nenhum suspeito foi identificado ou preso; cas...,São José do Rio Preto e Araçatuba,2024-04-03 22:26:33.756561,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/sp/sao-jose-do-rio-preto-...,Homem morto tiros peito boca perna Araçatuba
37009,Policial militar é assassinado a tiros enquant...,Agente estava a caminho do quartel quando foi ...,Ceará,2024-04-03 22:26:33.763455,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/ce/ceara/noticia/2024/04/...,Policial militar assassinado tiros trafegava m...
37010,Câmara de Taubaté aprova reajuste salarial de ...,Projeto foi aprovado por unanimidade pelos ver...,Vale do Paraíba e Região,2024-04-03 22:26:33.768573,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/sp/vale-do-paraiba-regiao...,Câmara Taubaté aprova reajuste salarial servi...
37011,Suspeito de atropelar ex-vereador no interior ...,Vítima de 74 anos foi atropelada ao atravessar...,Bahia,2024-04-03 22:26:33.773026,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/ba/bahia/noticia/2024/04/...,Suspeito atropelar ex-vereador interior BA apr...
37012,Programação do Arraiá do Interior 2024 é divul...,Atividades acontecerão de 1º de junho a 26 de ...,São João na Paraíba,2024-04-03 22:26:33.777132,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/pb/paraiba/sao-joao/notic...,Programação Arraiá Interior divulgada atraçõe...
37013,Dupla é presa com tijolos de cocaína e mais de...,Suspeitos foram detidos durante patrulhamento ...,Sorocaba e Jundiaí,2024-04-03 22:26:33.781281,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/sp/sorocaba-jundiai/notic...,Dupla presa tijolos cocaína R$ dinheiro Sorocaba
37014,VÍDEO: Mulher é assaltada dentro de carro a ca...,"Segundo o dono da empresa, a vítima do assalto...",Sul de Minas,2024-04-03 22:26:33.785399,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/mg/sul-de-minas/noticia/2...,VÍDEO Mulher assaltada carro agência bancária ...
37015,"Em Santarém, Sefa apreende mais de 66 mil garr...",Foram lavrados dois Termos de Apreensão e Depó...,Santarém e Região,2024-04-03 22:26:33.789469,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/pa/santarem-regiao/notici...,Em Santarém Sefa apreende garrafas bebida alc...
37016,Deputados autorizam empréstimo de R$ 400 milhõ...,Parlamentar da situação apontou que verba será...,Bahia,2024-04-03 22:26:33.793424,G1,https://s2-g1.glbimg.com/zDEEzOE01TTLfdu3jK5sd...,https://g1.globo.com/ba/bahia/noticia/2024/04/...,Deputados autorizam empréstimo R$ milhões Gov...


In [9]:
dataset_recomendations

,title,text,category,publication_date,source,image,link,normalized_data_scores,sentences
36204,Idoso de 74 anos tem casa invadida e é morto a...,"Segundo a polícia, vítima trabalhava como agio...",Piauí,2024-04-05 22:26:22.352815,G1,https://s2-g1.glbimg.com/y2pklZOHRvUmpvrToR6_U...,https://g1.globo.com/pi/piaui/noticia/2024/04/...,0.0,Idoso anos casa invadida morto tiros Parnaíba
36114,Barbeiro de 40 anos é morto a tiros na Região ...,Edmilson Almeida dos Santos foi assassinado no...,Bahia,2024-04-05 22:26:21.422101,G1,https://s2-g1.glbimg.com/2uukqWIP9YjkJ2IjYJD82...,https://g1.globo.com/ba/bahia/noticia/2024/04/...,0.0,Barbeiro anos morto tiros Região Metropolitan...
36590,Mossoró abre cadastramento para comerciantes d...,Cadastro para credrenciamento pode ser feito d...,Rio Grande do Norte,2024-04-05 22:26:27.309515,G1,https://s2-g1.glbimg.com/FongWEUSXVlhg5kVTKvWF...,https://g1.globo.com/rn/rio-grande-do-norte/no...,0.0,Mossoró abre cadastramento comerciantes Pingo ...
36054,Três pessoas são baleadas em caminhonete alvo ...,Vítimas foram atacadas por ocupantes de outro ...,Ceará,2024-04-05 22:26:20.735170,G1,https://s2-g1.glbimg.com/ywdPhNBKlSzbXJ1iWEnDj...,https://g1.globo.com/ce/ceara/noticia/2024/04/...,0.0,Três pessoas baleadas caminhonete alvo tiros F...
36302,Mototaxista é alvo de 15 tiros e morre em Mace...,Vítima aguardava chamada de clientes em ponto ...,Alagoas,2024-04-05 22:26:23.882668,G1,https://s2-g1.glbimg.com/8hTz9mZ07_l7WTzTBpZ_y...,https://g1.globo.com/al/alagoas/noticia/2024/0...,0.0,Mototaxista alvo tiros morre Maceió moto BMW ...
36389,"Durante atendimento de violência doméstica, Po...",Homem fugiu para um pasto próximo a residência...,Presidente Prudente e Região,2024-04-05 22:26:25.132145,G1,https://s2-g1.glbimg.com/Er4PCZw17CaNcwAO05Wmb...,https://g1.globo.com/sp/presidente-prudente-re...,0.0,Durante atendimento violência doméstica Políci...
36556,José Saud (Progressistas) sanciona reajuste sa...,O prefeito José Saud (Progressistas) é o autor...,Vale do Paraíba e Região,2024-04-05 22:26:26.969981,G1,https://s2-g1.glbimg.com/g4_maLftIB7lZg9gRBiqK...,https://g1.globo.com/sp/vale-do-paraiba-regiao...,0.0,José Saud Progressistas sanciona reajuste sala...
36569,Servidores municipais de Juiz de Fora têm reaj...,"Votação, em caráter extraordinário, aconteceu ...",Zona da Mata,2024-04-05 22:26:27.046139,G1,https://s2-g1.glbimg.com/QLtUzc24zMGasMHUgyq1G...,https://g1.globo.com/mg/zona-da-mata/noticia/2...,0.0,Servidores municipais Juiz Fora reajuste salar...
36378,"Sob protestos, reajuste dos servidores municip...",Projeto de Lei enviado pela prefeitura da capi...,Bahia,2024-04-05 22:26:24.979352,G1,https://s2-g1.glbimg.com/1YAGXkpe8qP_IHn6vAK4J...,https://g1.globo.com/ba/bahia/noticia/2024/04/...,0.0,Sob protestos reajuste servidores municipais a...
36500,Motorista preso duas vezes em menos de 24h por...,As duas situações ocorreram na noite dessa ter...,Mato Grosso,2024-04-05 22:26:26.159906,G1,https://s2-g1.glbimg.com/M-UZaeF9ktfaNE520WtFN...,https://g1.globo.com/mt/mato-grosso/noticia/20...,0.0,Motorista preso h atropelar motociclista fugir...
